In [ ]:
# . Make sure you have completed this example from lecture: Load data file cars.csv from Moodle
# a) Install Pandas profiler (see script example) and Great Expectations (you can also try Openrefine)
# b) Examine using Pandas profiler and Great Expectations. You can also test OpenRefine
# c) Do you find quality issues?
# d) Can they be fixed?


import pandas as pd
from ydata_profiling import ProfileReport

df = pd.read_csv("cars.csv")
df = df.drop(columns="index")
profile = ProfileReport(df, title="Profile report")
profile.to_file("./cars-profile.html")
profile


# Results missing ceels 1,3%
# Duplicate rows 14,5 %
# Engine volume missing data 4%

ModuleNotFoundError: No module named 'ydata_profiling'

In [ ]:
# Trying gret_expectations

import great_expectations as gx

# Get the Ephemeral Data Context (there are also other options)
context = gx.get_context()
assert type(context).__name__ == "EphemeralDataContext"

# Add a Pandas Data Source
data_source = context.data_sources.add_pandas(name="cars")

# Add a Data Asset to the Data Source
data_asset = data_source.add_dataframe_asset(name="cars_asset")

# Define the Batch Definition name
batch_definition_name = "cars_batch"

# Add the Batch Definition
batch_definition = data_asset.add_batch_definition_whole_dataframe(
    batch_definition_name
)
assert batch_definition.name == batch_definition_name

# Define the Batch Parameters
import pandas as pd

df = pd.read_csv("cars.csv")
batch_parameters = {"dataframe": df}

# Retrieve the Batch
batch = batch_definition.get_batch(batch_parameters=batch_parameters)

# Create an Expectation Suite
expectation_suite_name = "cars_suite"
suite = gx.ExpectationSuite(name=expectation_suite_name)

# Add Expectations
suite.add_expectation(
    gx.expectations.ExpectColumnDistinctValuesToBeInSet(
        column="Transmission",
        value_set=["1", "2", "3", "4", "5", "7"],  # must be these values
    )
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeUnique(column="index")  # must be unique
)

# Add the Expectation Suite to the Context
context.suites.add(suite)

In [14]:
# Validate the batch with the suite
validation_results = batch.validate(suite)

# Add missing run_id expected by the old renderer
if "meta" in validation_results and "run_id" not in validation_results["meta"]:
    validation_results["meta"]["run_id"] = "manual_run"

# Import necessary modules for rendering
from great_expectations.render.renderer import ValidationResultsPageRenderer
from great_expectations.render.view import DefaultJinjaPageView

# Initialize the renderer
renderer = ValidationResultsPageRenderer()

# Render the validation results
rendered = renderer.render(validation_results)

# Generate the HTML view
html = DefaultJinjaPageView().render(rendered)

with open("cars_validation_report.html", "w", encoding="utf-8") as f:
    f.write(html)

# distint values must belong to this set ["1", "2", "3", "4", "5", "7"], there are additional values observed "X" and "Y"
# all values in index are unique


NameError: name 'batch' is not defined

In [15]:
# 12. The data file titanic.csv contains a fraction of the Titanic passenger list, but some values are erroneous.
# a) What could be the validation criteria for each column? In other words, how the data in each column should be. Note that all the columns don't have very strong validation criteria and sometimes you have to investigate the data before defining the criteria.
# b) Investigate the data quality using Python Pandas Profiler, Openrefine and/or Great Expectations
# c) Describe the possible errors you found in each column and how you could fix them (if possible, no need to fix them)

import pandas as pd

df = pd.read_csv(
    "C:/Users/anast/OneDrive - Datu Analytics Oy/Personal/Assignment/Data engineering/streamlit_docker/titanic.csv"
)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Survived                 888 non-null    float64
 1   Pclass                   886 non-null    float64
 2   Name                     888 non-null    object 
 3   Sex                      886 non-null    object 
 4   Age                      886 non-null    float64
 5   Siblings/Spouses Aboard  889 non-null    int64  
 6   Parents/Children Aboard  884 non-null    float64
 7   Fare                     885 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 55.7+ KB


In [ ]:
# a) What could be the validation criteria for each column? In other words, how the data in each column should be. Note that all the columns don't have very strong validation criteria and sometimes you have to investigate the data before defining the criteria.
# 0   Survived                 888 non-null    float64  Validation_ 0 or 1
# 1   Pclass                   886 non-null    float64 Val: 1,2,3
# 2   Name                     888 non-null    object Val: Unique strinf
# 3   Sex                      886 non-null    object Val: female or male, string
# 4   Age                      886 non-null    float64 Val: Float between 0 and 120
# 5   Siblings/Spouses Aboard  889 non-null    int64  Val: Number between 0 and 20
# 6   Parents/Children Aboard  884 non-null    float64 Val: Number between 0 and 20
# 7   Fare                     885 non-null    float64 Val: Number between 0 and 5000

In [22]:
# b) Investigate the data quality using Python Pandas Profiler, Openrefine and/or Great Expectations
# c) Describe the possible errors you found in each column and how you could fix them (if possible, no need to fix them)

# Let's investigate using Profiler

from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Pofile report")
profile.to_file("/Titanic_25.11.2025.html")

ImportError: cannot import name 'Sentinel' from 'typing_extensions' (c:\Users\anast\AppData\Local\Programs\Python\Python310\lib\site-packages\typing_extensions.py)

In [ ]:
# Inverstigate the quality of great expecrations
# Lets use the code from leactures
import great_expectations as gx

# Get the Ephemeral Data Context
context = gx.get_context()
assert type(context).__name__ == "EphemeralDataContext"

# Add a Pandas Data Source
data_source = context.data_sources.add_pandas(name="titanic")

# Add a Data Asset to the Data Source
data_asset = data_source.add_dataframe_asset(name="titanic_asset")
# Define the Batch Definition name
batch_definition_name = "titanic_batch"

# Add the Batch Definition
batch_definition = data_asset.add_batch_definition_whole_dataframe(
    batch_definition_name
)
assert batch_definition.name == batch_definition_name

# Define the Batch Parameters
import pandas as pd

# Read the CSV file
df = pd.read_csv("./titanic.csv")

# Define the batch parameters
batch_parameters = {"dataframe": df}

# Retrieve the Batch
batch = batch_definition.get_batch(batch_parameters=batch_parameters)


# Create an Expectation Suite

suite.add_expectation(
    gx.expectations.ExpectColumnDistinctValuesToBeInSet(
        column="Survived", value_set=[0.0, 1.0]
    )
)

suite.add_expectation(
    gx.expectations.ExpectColumnDistinctValuesToBeInSet(
        column="Pclass", value_set=[1.0, 2.0, 3.0]
    )
)

suite.add_expectation(gx.expectations.ExpectColumnValuesToBeUnique(column="Name"))

suite.add_expectation(
    gx.expectations.ExpectColumnDistinctValuesToBeInSet(
        column="Sex", value_set=["female", "male"]
    )
)

suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="Age", min_value=0.0, max_value=120
    )
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="Siblings/Spouses Aboard", min_value=0.0, max_value=20
    )
)

suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="Parents/Children Aboard", min_value=0, max_value=20
    )
)

suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="Fare", min_value=0.0, max_value=5000
    )
)

# Add the Expectation Suite to the Context
context.suites.add(suite)


In [ ]:
# Validate the batch with the suite
validation_results = batch.validate(suite)

# Add missing run_id expected by the old renderer
if "meta" in validation_results and "run_id" not in validation_results["meta"]:
    validation_results["meta"]["run_id"] = "manual_run"

# Import necessary rendering classes
from great_expectations.render.renderer import ValidationResultsPageRenderer
from great_expectations.render.view import DefaultJinjaPageView

# Initialize the renderer
renderer = ValidationResultsPageRenderer()

# Render the validation results
rendered = renderer.render(validation_results)

# Generate the HTML view
html = DefaultJinjaPageView().render(rendered)

# Write the HTML to a file
with open("titanic_validation_report.html", "w", encoding="utf-8") as f:
    f.write(html)

In [ ]:
# Results
# Age vetween 0 and 120. Result all passed
# Fair between 0 and 5000. Restult 0,339 unexpected. Possible action to remove outliers
# Name must be unique 2 duplicates 0,45% unexpected, action to remove duplicates
# Parents two negative values or 0,22% remove the outliers
# Passenger class 1.0, 2.0, 3.0 some other values like 6.0, 22.0 and 33.0 are observed
# Sex are female and male, others are observed, replace to correct
# Sblings, everything as expected 0.0 and 1.0 but some other values are observed, can be reviewd and replaced

In [ ]:
# 13. Tempereture observations have some gaps.
# a) Investigate the fraction of missing data
import pandas as pd

df_temp = pd.read_csv(
    "C:/Users/anast/OneDrive - Datu Analytics Oy/Personal/Assignment/Data engineering/Temperature/Temperature_data_with_gaps.csv"
)
df_temp.head()
print("\n Number of missing datta in each column")
df_temp.isnull().sum()


 Number of missing datta in each column


Time             0
Temperature    758
dtype: int64

In [ ]:
# b) Fill the gaps using different interpolation methods
%matplotlib inline
df_temp["Time"] = pd.to_datetime(df_temp["Time"]) #correct format 
import matplotlib.pyplot as plt
import datetime

plt.figure(figsize=(12, 4))
plt.plot(df_temp["Time"], df_temp["Temperature"],interpolate(methods ='linear') #linear interpolation
plt.axis([datetime.date(2021,1,1),datetime.date(2021,2,1),-30,5]) #Zoom in

ModuleNotFoundError: No module named 'matplotlib.backends.registry'

In [ ]:
# c) Investigate how different methods perform statistically and compare results visually
# Tip: Pandas interpolate, methods: ['linear', 'time', 'index', 'values', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 'krogh', 'spline', 'polynomial', 'from_derivatives', 'piecewise_polynomial', 'pchip', 'akima', 'cubicspline']
# Example: df3['column'].interpolate(method='cubicspline', limit=10)

methods = [
    "linear",
    "time",
    "index",
    "values",
    "nearest",
    "zero",
    "slinear",
    "quadratic",
    "cubic",
    "barycentric",
    "krogh",
    "spline",
    "polynomial",
    "from_derivatives",
    "piecewise_polynomial",
    "pchip",
    "akima",
    "cubicspline",
]

df_lim = df_temp[0:100]  # limit the data

# loop all the methods

i = 1
for method in methods:
    plt.subplot(4, 4, i)
    plt.title(method)
    if method in ['spline', 'polynominal']:
        plt.plot(df_lim["Time"], df_lim["Temperature"], interpolate(methods=method, order = 3))  # different methods
        plt.plot(df_lim["Time"], df_lim["Temperature"])
    else
        plt.plot(df_lim["Time"], df_lim["Temperature"], interpolate(methods=method))  # different methods
        plt.plot(df_lim["Time"], df_lim["Temperature"])
    i = i + 1

In [ ]:
# Statistical comparison
methods = [
    "linear",
    "time",
    "index",
    "values",
    "nearest",
    "zero",
    "slinear",
    "quadratic",
    "cubic",
    "barycentric",
    "krogh",
    "spline",
    "polynomial",
    "from_derivatives",
    "piecewise_polynomial",
    "pchip",
    "akima",
    "cubicspline",
]
df_lim = df_temp[0:100]
# Loop through all the methods
print(
    "Original data with gaps. Mean:",
    df_lim["Temperature"].mean(),
    "Max value:",
    df_lim["Temperature"].max(),
    "Min value:",
    df_lim["Temperature"].min(),
)
for method in methods:
    if method in ["spline", "polynomial"]:
        interpolate_data = df_lim["Temperature"].interpolate(
            method=method, order=3, limit=5
        )
    else:
        interpolate_data = df_lim["Temperature"].interpolate(method=method, limit=5)
    print(
        method,
        "interpolation. Mean:",
        interpolate_data.mean(),
        "Max value:",
        interpolate_data.max(),
        "Min value:",
        interpolate_data.min(),
    )


Original data with gaps. Mean: -8.620000000000001 Max value: -0.9 Min value: -19.2
linear interpolation. Mean: -8.779983457402812 Max value: -0.9 Min value: -19.2


ValueError: time-weighted interpolation only works on Series or DataFrames with a DatetimeIndex